In [1]:
import os

import nibabel  # for loading and saving nifty files
import torch
from math import ceil, floor
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import Pad
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from nibabel.affines import from_matvec, to_matvec, apply_affine

from skimage import io
import matplotlib.pyplot as plt

from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff
import ants

### preprocessing steps for KORA

1. rescale to positive intensity values (needed for N4 correction), so far just add 10, save original values in .pkl file
2. bias field correction (N4) compute on in-phase scan (external script)
3. apply in-phase bias field to opp scans
4. rescale intensities back to original values (after bias phase correction, the intensities are close to 0 → second N4 won’t work if we don’t rescale the intensities)

Now for in and opp scans:
5. change orientation to RAS 
6. resampling of lower scan to the resolution of upper scan
7. merging of upper and lower scan
8. rescale to positive intensity values (needed for N4 correction)
9. compute bias field correction on merged in-phase scans (external script)
10. apply bias field correction to merged opp scans

missing: intensity standardization across all KORA scans, cropping of scans to a common FOV & size

Segmentations:
11. merging of segmentation files and changing of labels to 0:7
12. optional: resampling to 2,2,3mm (only needed when we want to create a KORANAKOUKB dataset)

intensity standardization across datasets? (KORA, NAKO, UKB)


Some problems: so far running this on rtx locally, getting corrupted files when running directly on nas. why?


### Step 1. rescale to positive intensity values of iso_in scans (needed for N4 correction)

In [10]:
# save 2 lists of original filenames and original (min,max values) - to use for rescaling the n4 corrected scans back
# to their original intensity range

original_values_dict = {}

In [24]:
def rescale_intensities(img_files, data_dir,sequence):
    images = []
    filenames = []
    values = []
    original_filenames = [[],[]]
    original_values = [[],[]]
    _, pat_id = os.path.split(data_dir)
    print('Pat ID ', pat_id)
    if sequence == 'in':
        print('create entry in dict for ', pat_id)
        original_values_dict[pat_id] = {}
    original_values_dict[pat_id][sequence] = {}
    original_values_dict[pat_id][sequence]['filenames'] = []
    original_values_dict[pat_id][sequence]['min_max_values'] = []

    # load files
    for file in img_files:
        if "nii.gz" in file:
            img = nibabel.load(os.path.join(data_dir, file))
            print('filename: ', file)
            filename, file_extension = os.path.splitext(file)
            filename, file_extension = os.path.splitext(filename)
            original_values_dict[pat_id][sequence]['filenames'].append(filename)

            images.append(img)
            img_data = np.asanyarray(img.dataobj)
            min_value = np.min(img_data)
            max_value = np.max(img_data)
            print('dtype ', img_data.dtype)
            print('img.get_data_dtype() ', img.get_data_dtype())
            print('header dtpye ', img.header['datatype'])
            print('header bitpix ', img.header['bitpix'])
            #print(img.header)
            print('min value: ', min_value )
            print('max value: ', max_value)
            
            if min_value < 0:
                print('Negative values detected!!')
            else:
                new_img_data = img_data + 10
            min_value = np.min(new_img_data)
            max_value = np.max(new_img_data)
            print('new min value: ', min_value )
            print('new max value: ', max_value)
            print('new image dtype ', new_img_data.dtype)
            original_values_dict[pat_id][sequence]['min_max_values'].append([min_value,max_value])

            new_img_nii = nibabel.Nifti1Image(new_img_data, img.affine.copy(), img.header.copy())
            new_file_path = os.path.join(data_dir,filename + '_rescaled.nii.gz')
            nibabel.save(new_img_nii, new_file_path)

In [ ]:
data_dir = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/'

listFolders =  os.listdir(data_dir)
for folder in listFolders:
    print(folder)
    path = data_dir + folder
    listFiles = os.listdir(path)
    sequence = 'in'
    in_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and 'corrected' not in f and not'resampled' in f and not 'rescaled' in f ]
    print(in_files)

    rescale_intensities(in_files, path,sequence)
    
    sequence = 'opp'
    opp_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and 'corrected' not in f and not'resampled' in f and not 'rescaled' in f ]
    print(opp_files)

    rescale_intensities(opp_files, path,sequence)
    

In [25]:
# if sth went wrong can run it here again for single patient
path = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/KORA2452879'
listFiles = os.listdir(path)
sequence = 'in'
in_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and 'corrected' not in f and not'resampled' in f and not 'rescaled' in f ]
print(in_files)

rescale_intensities(in_files, path,sequence)
sequence = 'opp'
opp_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and 'corrected' not in f and not'resampled' in f and not 'rescaled' in f ]
print(opp_files)

rescale_intensities(opp_files, path,sequence)

['t1_vibe_dixon_cor_caipi6_bh_288_iso_in_6.nii.gz', 't1_vibe_dixon_cor_caipi6_bh_288_iso_in_7.nii.gz']
Pat ID  KORA2452879
create entry in dict for  KORA2452879
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_in_6.nii.gz
dtype  float32
img.get_data_dtype()  float32
header dtpye  16
header bitpix  32
min value:  0.0
max value:  895.0
new min value:  10.0
new max value:  905.0
new image dtype  float32
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_in_7.nii.gz
dtype  float32
img.get_data_dtype()  float32
header dtpye  16
header bitpix  32
min value:  0.0
max value:  548.0
new min value:  10.0
new max value:  558.0
new image dtype  float32
['t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9.nii.gz', 't1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz']
Pat ID  KORA2452879
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9.nii.gz
dtype  float32
img.get_data_dtype()  float32
header dtpye  16
header bitpix  32
min value:  0.0
max value:  524.0
new min value:  10.0
new max value:  534.0
new image dtype

In [ ]:
import csv
import pickle
print(original_values_dict)
file = open('original_values_dict.pkl', 'wb')
pickle.dump(original_values_dict, file)
file.close()

### Step 2. Run N4 bias field correction on rescaled in scans. (script n4.sh under /home/anne/whole_body_segmentation

### Step 3. load bias field masks of in scans and apply them to opp scans

In [26]:
def apply_bias_fields(img_files, bias_fields, data_dir,sequence):
    images = []
    bias_f = []
    filenames = []
    # load files
    for file in img_files:
        if "nii.gz" in file:
            img = nibabel.load(os.path.join(data_dir, file))
            print('filename: ', file)
            filename, file_extension = os.path.splitext(file)
            filename, file_extension = os.path.splitext(filename)
            
            images.append(img)
            filenames.append(filename)
    for file in bias_fields:
        if "nii.gz" in file:
            img = nibabel.load(os.path.join(data_dir, file))
            print('filename: ', file)            
            bias_f.append(img)

    images_sorted, filenames_sorted = zip(*sorted(zip(images,filenames), key=lambda im: im[0].header['qoffset_z'], reverse=True))
    bias_f = sorted(bias_f, key=lambda im: im.header['qoffset_z'], reverse=True)
    
    im_0 = images_sorted[0]
    im_1 = images_sorted[1]
    
    bias_0 = bias_f[0]
    bias_1 = bias_f[1]
    
    # manually apply bias field to image:
    im_0_data = np.asanyarray(im_0.dataobj)
    im_1_data = np.asanyarray(im_1.dataobj)
    print('min ', np.min(im_0_data))
    print('max ', np.max(im_0_data))
    bias_0_data = bias_0.get_fdata()
    bias_1_data = bias_1.get_fdata()
    print('min ', np.min(bias_0_data))
    print('max ', np.max(bias_0_data))
    
    corrected_0 = im_0_data / bias_0_data
    corrected_1 = im_1_data / bias_1_data
    print('min ', np.min(corrected_0))
    print('max ', np.max(corrected_0))

    new_img_nii = nibabel.Nifti1Image(corrected_0, im_0.affine, im_0.header)
    new_file_path = os.path.join(data_dir,filenames_sorted[0] + '_applied_in_bias_corrected.nii.gz')
    nibabel.save(new_img_nii, new_file_path)
    
    new_img_nii = nibabel.Nifti1Image(corrected_1, im_1.affine, im_1.header)
    new_file_path = os.path.join(data_dir,filenames_sorted[1] + '_applied_in_bias_corrected.nii.gz')
    nibabel.save(new_img_nii, new_file_path)

In [ ]:
data_dir = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/'

listFolders =  os.listdir(data_dir)
for folder in listFolders:
    print(folder)
    path = data_dir + folder
    listFiles = os.listdir(path)
    # sequence for bias fields is 'in'
    sequence = 'in'
    bias_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and 'corrected' not in f and not'resampled' in f and 'rescaled' in f and 'bias' in f ]
    print(bias_files)
    # want to apply in bias fields on opp scans, so load opp scans here:
    sequence = 'opp'
    opp_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and 'corrected' not in f and not'resampled' in f and 'rescaled' in f and not 'bias' in f ]
    print(opp_files)
    apply_bias_fields(opp_files, bias_files, path,sequence)

In [27]:
# if sth went wrong can run it here again for single patient
path = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/KORA2452879'
listFiles = os.listdir(path)
sequence = 'in'
bias_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and 'corrected' not in f and not'resampled' in f and 'rescaled' in f and 'bias' in f ]
print(bias_files)
    
sequence = 'opp'
opp_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and 'corrected' not in f and not'resampled' in f and 'rescaled' in f and not 'bias' in f ]
print(opp_files)
apply_bias_fields(opp_files, bias_files, path,sequence)

['t1_vibe_dixon_cor_caipi6_bh_288_iso_in_7_rescaled_bias_field.nii.gz', 't1_vibe_dixon_cor_caipi6_bh_288_iso_in_6_rescaled_bias_field.nii.gz']
['t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_rescaled.nii.gz', 't1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_rescaled.nii.gz']
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_rescaled.nii.gz
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_rescaled.nii.gz
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_in_7_rescaled_bias_field.nii.gz
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_in_6_rescaled_bias_field.nii.gz
min  10.0
max  889.0
min  72.12023162841797
max  525.4364624023438
min  0.019736438320949973
max  5.259675841356089


### Step 4 Rescale images back to their original intensity range

In [33]:
def rescale_intensities(img_files, data_dir,sequence):
    images = []
    filenames = []
    _,pat_id = os.path.split(data_dir)
    # load files
    for file in img_files:
        if "nii.gz" in file:
            img = nibabel.load(os.path.join(data_dir, file))
            print('filename: ', file)
            filename, file_extension = os.path.splitext(file)
            filename, file_extension = os.path.splitext(filename)
            filenames.append(filename)
            images.append(img)
            img_data = img.get_fdata()
            
    images_sorted, filenames_sorted = zip(*sorted(zip(images,filenames), key=lambda im: im[1], reverse=True))
    original_values_dict[pat_id][sequence]['filenames'], original_values_dict[pat_id][sequence]['min_max_values'] = zip(*sorted(zip(original_values_dict[pat_id][sequence]['filenames'],original_values_dict[pat_id][sequence]['min_max_values']), key=lambda im: im[0], reverse=True))

    for i, img in enumerate(images_sorted):
        orig_f = original_values_dict[pat_id][sequence]['filenames'][i]
        orig_v = original_values_dict[pat_id][sequence]['min_max_values'][i]
        name = filenames_sorted[i]
        
        print('name: ', name)
        print('orig name: ', orig_f)
        print('orig values ', orig_v)
        
        img_data = img.get_fdata()
        min_value = np.min(img_data)
        max_value = np.max(img_data)
        print('img min: ', min_value)
        print('img max: ', max_value)
        
        img_data = (img_data - min_value) * (orig_v[1] - orig_v[0])/(max_value - min_value) + orig_v[0]
        
        print('new min: ', np.min(img_data))
        print('new max: ', np.max(img_data))
        
        new_img_nii = nibabel.Nifti1Image(img_data, img.affine, img.header)
        new_file_path = os.path.join(data_dir,name + '_rescaled_original.nii.gz')
        nibabel.save(new_img_nii, new_file_path)

In [ ]:
data_dir = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/'

listFolders =  os.listdir(data_dir)
for folder in listFolders:
    print(folder)
    path = data_dir + folder
    listFiles = os.listdir(path)
    sequence = 'in'
    in_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'rescaled_corrected' in f and 'bias' not in f ]
    print(in_files)

    rescale_intensities(in_files, path,sequence)
    
    sequence = 'opp'
    opp_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'rescaled_applied_in_bias_corrected' in f ]
    print(opp_files)

    rescale_intensities(opp_files, path,sequence)
    

In [34]:
# if sth went wrong can run it here again for single patient
path = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/KORA2452879'
listFiles = os.listdir(path)

    
sequence = 'opp'
opp_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'rescaled_applied_in_bias_corrected.nii.gz' in f ]
print(opp_files)
rescale_intensities(opp_files, path,sequence)

['t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_rescaled_applied_in_bias_corrected.nii.gz', 't1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_rescaled_applied_in_bias_corrected.nii.gz']
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_rescaled_applied_in_bias_corrected.nii.gz
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_rescaled_applied_in_bias_corrected.nii.gz
name:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_rescaled_applied_in_bias_corrected
orig name:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9
orig values  [10.0, 534.0]
img min:  0.014345293864607811
img max:  2.8343071937561035
new min:  10.0
new max:  534.0
name:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_rescaled_applied_in_bias_corrected
orig name:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8
orig values  [10.0, 889.0]
img min:  0.019736438989639282
img max:  5.259675979614258
new min:  10.0
new max:  889.0


### Step 5+6: Reorient to RAS and resample lower scan to upper scan resolution

In [30]:
from nibabel.processing import resample_to_output, resample_from_to
def reorient_and_resample_images(img_files, data_dir, sequence, target_voxel_dim):
    images = []
    filenames = []
    # load files
    for file in img_files:
        if "nii.gz" in file:
            img = nibabel.load(os.path.join(data_dir, file))
            print('filename: ', file)
            filename, file_extension = os.path.splitext(file)
            filename, file_extension = os.path.splitext(filename)
            filenames.append(filename)
            
            print('min value: ', np.min(img.get_fdata()))
            print('max value: ', np.max(img.get_fdata()))
            #print(img_h.get_data_shape())
            print('original orientation: ', nibabel.orientations.aff2axcodes(img.header.get_best_affine()))

            # convert to RAS so that this code works:
            img = do_nibabel_transform_to_ras(img)
            print('min value: ', np.min(img.get_fdata()))
            print('max value: ', np.max(img.get_fdata()))
            images.append(img)
            
            img_h = img.header

            steps = img_h['pixdim'][1:4]
            

    images_sorted, filenames_sorted = zip(*sorted(zip(images,filenames), key=lambda im: im[0].header['qoffset_z'], reverse=True))
    #print(sorted(zip(images,filenames), key=lambda im: im[0].header['qoffset_z'], reverse=True))
    img_0 = images_sorted[0]
    img_1 = images_sorted[1]
    
    print('im 0 affine: ', img_0.header.get_best_affine())
    print('im 1 affine: ', img_1.header.get_best_affine())

    target_affine = img_0.header.get_best_affine()
    target_affine[2,3] = img_1.header.get_best_affine()[2,3]
    img_1 = resample_from_to(img_1, [img_1.shape, target_affine])
    print('min value: ', np.min(img_1.get_fdata()))
    print('max value: ', np.max(img_1.get_fdata()))
    new_file_path = os.path.join(data_dir,filenames_sorted[1]+'_resampled.nii.gz')

    nibabel.save(img_1, new_file_path)
    
    new_file_path = os.path.join(data_dir,filenames_sorted[0]+'_resampled.nii.gz')

    nibabel.save(img_0, new_file_path)
 

In [31]:
def do_nibabel_transform_to_ras(img):
    ###### use nibabel transform
    affine = img.get_affine()
    orig_ornt = nibabel.io_orientation(affine)
    #print('orig image orientation: ', orig_ornt)
    #print('transform to RAS')
    targ_ornt = axcodes2ornt('RAS')
    #print('target orientation: ', targ_ornt)
    transform = ornt_transform(orig_ornt, targ_ornt)
    #fat_tmp = fat.get_data()
    img = img.as_reoriented(transform)
    #print('img shape after nibabel transform: ', img.get_data().shape)

    #########################
    return img

In [ ]:
data_dir = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/'

listFolders =  os.listdir(data_dir)
for folder in listFolders:
    print(folder)
    path = data_dir + folder
    listFiles = os.listdir(path)
    sequence = 'opp'
    # with previous bias field correction
    opp_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and '_applied_in_bias_corrected_rescaled_original' in f and not 'resampled' in f ]
    # without previous bias field correction
    #opp_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and not '_applied_in_bias_corrected' in f and not 'resampled' in f ]

    print(opp_files)
    print('first reorient and resample')
    reorient_and_resample_images(opp_files, path, sequence, [2,2,3])
    
    sequence = 'in'
    in_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and '_rescaled_corrected_rescaled_original' in f and not 'resampled' in f ]
    # without previous bias field correction
    #in_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and not '_rescaled_corrected' in f and not 'resampled' in f ]

    print(in_files)
    print('first reorient and resample')
    reorient_and_resample_images(in_files, path, sequence, [2,2,3])
    
    

In [35]:
# if sth went wrong can run it here again for single patient
path = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/KORA2452879'
listFiles = os.listdir(path)

    
sequence = 'opp'
opp_files = [f for f in listFiles if sequence in f and 'COMP' not in f and 'comb' not in f and '_applied_in_bias_corrected_rescaled_original' in f and not 'resampled' in f ]
print(opp_files)
reorient_and_resample_images(opp_files, path,sequence, [2,2,3])

['t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_rescaled_applied_in_bias_corrected_rescaled_original.nii.gz', 't1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_rescaled_applied_in_bias_corrected_rescaled_original.nii.gz']
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_rescaled_applied_in_bias_corrected_rescaled_original.nii.gz
min value:  10.0
max value:  534.0
original orientation:  ('L', 'I', 'P')


/home/anne/anaconda3/envs/mesh_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning:

get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0



min value:  10.0
max value:  534.0
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_rescaled_applied_in_bias_corrected_rescaled_original.nii.gz
min value:  10.0
max value:  889.0
original orientation:  ('L', 'I', 'P')
min value:  10.0
max value:  889.0
im 0 affine:  [[   1.70138884    0.            0.         -243.27729797]
 [   0.            1.69999695    0.         -151.14399719]
 [   0.            0.            1.70138884 -468.50531006]
 [   0.            0.            0.            1.        ]]
im 1 affine:  [[   1.66666663    0.            0.         -238.31202698]
 [   0.            1.69999695    0.         -151.14399719]
 [   0.            0.            1.66666663 -693.54003906]
 [   0.            0.            0.            1.        ]]
min value:  -9.355963706970215
max value:  540.888916015625


### Step 7: Merging of upper and lower scan

In [36]:
def merge_images(opp_files, data_dir, sequence):
    
    images = []
    filenames = []
    # load files
    for file in opp_files:
        if "nii.gz" in file:
            img = nibabel.load(os.path.join(data_dir, file))
            print('filename: ', file)
            filename, file_extension = os.path.splitext(file)
            filename, file_extension = os.path.splitext(filename)
            images.append(img)
            
            img_h = img.header
            print('x: ', img_h['qoffset_x'])
            print('y: ', img_h['qoffset_y'])
            print('z: ', img_h['qoffset_z'])
            
            print('min value: ', np.min(img.get_fdata()))
            print('max value: ', np.max(img.get_fdata()))
            filenames.append(filename)
            

    
   
    images_sorted, filenames_sorted = zip(*sorted(zip(images,filenames), key=lambda im: im[0].header['qoffset_z'], reverse=True))
 
    #take the upper 2 scans
    im_0 = images_sorted[0]
    im_1 = images_sorted[1]

    im_0_dim_v = im_0.shape[2]
    im_1_dim_v = im_1.shape[2]

    #print(im_0_dim_v)

    # calculate overlap region:
    im_0_end = im_0.header['qoffset_z']
    im_1_end = im_1.header['qoffset_z']
    print('im_0_end, im_1,end: ', im_0_end, im_1_end)

    spacing = im_0.header['pixdim'][3]
    print('pixel spacing: ', spacing)

    im_0_dim_w = im_0_dim_v * spacing
    im_1_dim_w = im_1_dim_v * spacing

    im_1_start = im_1_end + im_1_dim_w
    im_0_start = im_0_end + im_0_dim_w
    print('im_0_start, im_1,start: ', im_1_start, im_0_start)

    overlap = abs(im_0_end - im_1_start)
    print('overlap: ', overlap)

    overlap_v = int(round(overlap/spacing))
    print('overlap_v: ', overlap_v)

    new_im_dim = round((abs(im_1_end - im_0_end) + abs(im_0_end - im_0_start))/spacing)
    print('new im dim: ', new_im_dim)
    
    new_img = np.empty([im_0.shape[0], im_0.shape[1], int(new_im_dim)])
    print('new img shape: ', new_img.shape)
    im_0_data = im_0.get_data()
    im_1_data = im_1.get_data()


    # bottom ( origin is bottom left)    
    new_img[:,:,0:(im_1_dim_v-overlap_v)] = im_1_data[:,:,0:(im_1_dim_v-overlap_v)]    
    # top:
    new_img[:,:, im_1_dim_v:] = im_0_data[:,:,overlap_v:]
    
    # overlap region:
    a = 1/overlap_v
    print('overlap_v ', overlap_v)
    print('a ', a)
    sig = sigmoid(overlap_v)
    for l in range(0,overlap_v):
        #new_img[:,:,(im_1_dim_v-overlap_v+l)] = (1-a*l) * im_1_data[:,:,(im_1_dim_v-overlap_v)+l] + (a*l) * im_0_data[:,:,l]
        new_img[:,:,(im_1_dim_v-overlap_v+l)] = (1-sig[l]) * im_1_data[:,:,(im_1_dim_v-overlap_v)+l] + (sig[l]) * im_0_data[:,:,l]

    empty_header = nibabel.Nifti1Header()
    new_img_nii = nibabel.Nifti1Image(new_img, im_1.affine, empty_header)
    new_img_nii.header['pixdim'] = im_1.header['pixdim']

    #print('new header: ', new_img_nii.header)
    new_file_path = os.path.join(data_dir,'t1_vibe_dixon_cor_caipi6_bh_288_iso_%s_corrected_comb_sigm.nii.gz'%sequence)

    nibabel.save(new_img_nii, new_file_path)
    

In [37]:
import numpy as np 
import math 
import matplotlib.pyplot as plt 

def sigmoid(v_overlap):
    x = np.linspace(0, v_overlap, v_overlap)
    k = 0.5
    x0 = v_overlap//2
    z = 1/(1 + np.exp(-k*(x-x0))) 
    #print(z)

    #plt.plot(x, z) 
    #plt.xlabel("x") 
    #plt.ylabel("Sigmoid(X)") 

    #plt.show() 
    return z

In [ ]:
data_dir = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/'

listFolders =  os.listdir(data_dir)
for folder in listFolders:
    print(folder)
    path = data_dir + folder
    listFiles = os.listdir(path)
    sequence = 'opp'
    opp_files = [f for f in listFiles if sequence in f and  '_applied_in_bias_corrected_rescaled_original_resampled' in f ]
    # without previous bias field correction
    #opp_files = [f for f in listFiles if sequence in f and not 'comb' in f and not'_applied_in_bias_corrected' in f and 'resampled' in f]

    print(opp_files)
    merge_images(opp_files, path,sequence)
    
    sequence = 'in'
    
    in_files = [f for f in listFiles if sequence in f and  '_rescaled_original_resampled' in f and 'opp' not in f ]
    # without previous bias field correction
    #in_files = [f for f in listFiles if sequence in f and not 'comb' in f and not '_rescaled_corrected' in f and 'resampled' in f]

    print(in_files)
    merge_images(in_files, path,sequence)

In [38]:
# if sth went wrong can run it here again for single patient
path = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/KORA2452879'
listFiles = os.listdir(path)

    
sequence = 'opp'
opp_files = [f for f in listFiles if sequence in f and  '_applied_in_bias_corrected_rescaled_original_resampled' in f ]
print(opp_files)
merge_images(opp_files, path,sequence)

['t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_rescaled_applied_in_bias_corrected_rescaled_original_resampled.nii.gz', 't1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_rescaled_applied_in_bias_corrected_rescaled_original_resampled.nii.gz']
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_rescaled_applied_in_bias_corrected_rescaled_original_resampled.nii.gz
x:  -243.2773
y:  -151.144
z:  -468.5053
min value:  10.0
max value:  889.0
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9_rescaled_applied_in_bias_corrected_rescaled_original_resampled.nii.gz
x:  -243.2773
y:  -151.144
z:  -693.54004
min value:  -9.355963706970215
max value:  540.888916015625
im_0_end, im_1,end:  -468.5053 -693.54004
pixel spacing:  1.7013888
im_0_start, im_1,start:  -203.54005432128906 21.494674682617188
overlap:  264.9652557373047
overlap_v:  156
new im dim:  420.0
new img shape:  (288, 160, 420)


/home/anne/anaconda3/envs/mesh_env/lib/python3.6/site-packages/ipykernel_launcher.py:63: DeprecationWarning:

get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0

/home/anne/anaconda3/envs/mesh_env/lib/python3.6/site-packages/ipykernel_launcher.py:64: DeprecationWarning:

get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0



overlap_v  156
a  0.00641025641025641


### Step 8: rescale intensities for second N4 correction

In [39]:
def rescale_intensities(img_files, data_dir,sequence):
    images = []
    # load files
    for file in img_files:
        if "nii.gz" in file:
            img = nibabel.load(os.path.join(data_dir, file))
            print('filename: ', file)
            filename, file_extension = os.path.splitext(file)
            filename, file_extension = os.path.splitext(filename)
            
            images.append(img)
            img_data = img.get_fdata()
            min_value = np.min(img_data)
            max_value = np.max(img_data)
            print('min value: ', min_value )
            print('max value: ', max_value)
            
            if min_value < 0 and abs(min_value) >= 10:
                print('Negative values detected!!')                
                img_data = img_data + abs(min_value) +10

            else:
                img_data = img_data + 10
            min_value = np.min(img_data)
            max_value = np.max(img_data)
            print('new min value: ', min_value )
            print('new max value: ', max_value)
            
            new_img_nii = nibabel.Nifti1Image(img_data, img.affine, img.header)
            new_file_path = os.path.join(data_dir,filename + '_rescaled.nii.gz')
            nibabel.save(new_img_nii, new_file_path)

In [ ]:
data_dir = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/'

listFolders =  os.listdir(data_dir)
for folder in listFolders:
    print(folder)
    path = data_dir + folder
    listFiles = os.listdir(path)
    sequence = 'in'
    in_files = [f for f in listFiles if sequence in f and 'corrected_comb_sigm' in f and 'rescaled' not in f ]
    print(in_files)
    rescale_intensities(in_files, path,sequence)
    
    sequence = 'opp'
    opp_files = [f for f in listFiles if sequence in f and 'corrected_comb_sigm' in f and 'rescaled' not in f ]
    print(opp_files)
    rescale_intensities(opp_files, path,sequence)
    

In [40]:
path = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/KORA2452879'
listFiles = os.listdir(path)

    
sequence = 'opp'
opp_files = [f for f in listFiles if sequence in f and 'corrected_comb_sigm' in f and 'rescaled' not in f ]
print(opp_files)
rescale_intensities(opp_files, path,sequence)

['t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_corrected_comb_sigm.nii.gz']
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_corrected_comb_sigm.nii.gz
min value:  -9.355963706970215
max value:  889.0
new min value:  0.6440362930297852
new max value:  899.0


### Step 9: run N4 correction script on in comb files

### Step 10: apply bias field of in scans on opp scans

In [41]:
def apply_bias_fields(img_files, bias_fields, data_dir,sequence):
    images = []
    bias_f = []
    filenames = []
    # load files
    for file in img_files:
        if "nii.gz" in file:
            img = nibabel.load(os.path.join(data_dir, file))
            print('filename: ', file)
            filename, file_extension = os.path.splitext(file)
            filename, file_extension = os.path.splitext(filename)
            
            images.append(img)
            filenames.append(filename)
    for file in bias_fields:
        if "nii.gz" in file:
            img = nibabel.load(os.path.join(data_dir, file))
            print('filename: ', file)            
            bias_f.append(img)

    images_sorted, filenames_sorted = zip(*sorted(zip(images,filenames), key=lambda im: im[0].header['qoffset_z'], reverse=True))
    bias_f = sorted(bias_f, key=lambda im: im.header['qoffset_z'], reverse=True)
    
    im_0 = images_sorted[0]
    
    bias_0 = bias_f[0]
    
    # manually apply bias field to image:
    im_0_data = im_0.get_fdata()
    print('min ', np.min(im_0_data))
    print('max ', np.max(im_0_data))
    bias_0_data = bias_0.get_fdata()
    print('min ', np.min(bias_0_data))
    print('max ', np.max(bias_0_data))
    
    corrected_0 = im_0_data / bias_0_data
    print('min ', np.min(corrected_0))
    print('max ', np.max(corrected_0))

    new_img_nii = nibabel.Nifti1Image(corrected_0, im_0.affine, im_0.header)
    new_file_path = os.path.join(data_dir,filenames_sorted[0] + '_applied_in_bias_corrected.nii.gz')
    nibabel.save(new_img_nii, new_file_path)
    

In [ ]:
data_dir = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/'

listFolders =  os.listdir(data_dir)
for folder in listFolders:
    print(folder)
    path = data_dir + folder
    listFiles = os.listdir(path)
    # sequence for bias fields is 'in'
    sequence = 'in'
    bias_files = [f for f in listFiles if sequence in f and '_comb_sigm_rescaled_bias_field' in f ]
    print(bias_files)
    # want to apply in bias fields on opp scans, so load opp scans here:
    sequence = 'opp'
    opp_files = [f for f in listFiles if sequence in f and '_comb_sigm_rescaled.nii.gz' in f ]
    print(opp_files)
    apply_bias_fields(opp_files, bias_files, path,sequence)

In [42]:
path = '/home/anne/whole_body_segmentation/All_KORA_original_data/All/KORA2452879'
listFiles = os.listdir(path)

    
# sequence for bias fields is 'in'
sequence = 'in'
bias_files = [f for f in listFiles if sequence in f and '_comb_sigm_rescaled_bias_field' in f ]
print(bias_files)
# want to apply in bias fields on opp scans, so load opp scans here:
sequence = 'opp'
opp_files = [f for f in listFiles if sequence in f and '_comb_sigm_rescaled.nii.gz' in f ]
print(opp_files)
apply_bias_fields(opp_files, bias_files, path,sequence)

['t1_vibe_dixon_cor_caipi6_bh_288_iso_in_corrected_comb_sigm_rescaled_bias_field.nii.gz']
['t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_corrected_comb_sigm_rescaled.nii.gz']
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_corrected_comb_sigm_rescaled.nii.gz
filename:  t1_vibe_dixon_cor_caipi6_bh_288_iso_in_corrected_comb_sigm_rescaled_bias_field.nii.gz
min  0.6440362930297852
max  899.0
min  92.43206787109375
max  340.2630615234375
min  0.006357685659736475
max  5.4364844628675115
